In [1]:
import nest_asyncio

nest_asyncio.apply()

In [8]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import pandas as pd
import os
import pathlib

root_dir = pathlib.PurePath(os.path.dirname(os.getcwd())).parent
data_dir = os.path.join(root_dir, 'data')

## Custom Prompt를 사용하는 방법을 알아봅시다.

Human in the loop의 핵심 => Custom Prompt를 활용해 보다 좋은 질문 생성법을 만드는 방식

In [3]:
from llama_index.core.base.llms.types import ChatMessage, MessageRole
from autorag.data.qa.query.llama_gen_query import llama_index_generate_base
from llama_index.core.base.llms.base import BaseLLM
from typing import Dict


async def custom_query_gen(
		row: Dict,
		llm: BaseLLM,
		lang: str = "en",
) -> Dict:
	CUSTOM_PROMPT = {
		"ko": [
			ChatMessage(
				role=MessageRole.SYSTEM,
				content="""
주어진 정보를 바탕으로 질문을 생성하세요.			
""",
			)
		]
	}

	return await llama_index_generate_base(
		row, llm, CUSTOM_PROMPT[lang]
	)

[01/12/25 16:58:51] INFO     [__init__.py:100] >> You are using API version of AutoRAG.To use local ]8;id=879324;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/autorag/__init__.py\__init__.py]8;;\:]8;id=40983;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/autorag/__init__.py#100\100]8;;\
                             version, run pip install 'AutoRAG[gpu]'                                               

                    INFO     [__init__.py:128] >> You are using API version of AutoRAG.To use local ]8;id=799483;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/autorag/__init__.py\__init__.py]8;;\:]8;id=744970;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/autorag/__init__.py#128\128]8;;\
                             version, run pip install 'AutoRAG[gpu]'                                               

In [5]:
corpus_data_path = os.path.join(data_dir, 'data_creation', 'corpus.parquet')
corpus_df = pd.read_parquet(corpus_data_path, engine='pyarrow')

In [ ]:
from autorag.data.qa.sample import random_single_hop
from autorag.data.qa.schema import Corpus

initial_corpus = Corpus(corpus_df)
initial_qa = initial_corpus.sample(random_single_hop, n=3).map(
        lambda df: df.reset_index(drop=True),
    ).make_retrieval_gt_contents()


Custom Prompt를 적용합니다

In [9]:
from llama_index.llms.openai import OpenAI

llm = OpenAI()
first_qa = initial_qa.batch_apply(
	custom_query_gen,
	llm=llm,
	lang="ko",
)

[01/12/25 17:02:08] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=976157;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=285460;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[01/12/25 17:02:09] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=397510;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=637662;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=620723;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=563797;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

In [11]:
first_qa.data['query']

0                    해외VC로 투자를 유치하기 위한 조건 중 하나는 무엇인가요?
1    어떤 분야의 기업들이 해외 VC 투자를 받을 수 있는 조건이나 지원을 받을 수 있는...
2                       창업기업 선정평가의 절차는 몇 단계로 이루어져 있나요?
Name: query, dtype: object

retrieval gt에 알맞은 질문이 생성되었는지 확인해 봅니다.

In [21]:
def visualize_query_retrieval_gt(df, idx: int):
	print(f'질문 : {df.data.iloc[idx]["query"]}')
	print(f"단락 내용:\n{df.data.iloc[idx]['retrieval_gt_contents'][0][0]}")

In [22]:
visualize_query_retrieval_gt(first_qa, 0)

질문 : 해외VC로 투자를 유치하기 위한 조건 중 하나는 무엇인가요?
단락 내용:
본점과 지점이 있는 경우 본점 사업자로만 신청 가능하며, 중복 신청한 경우 본점을 기준으로
요건검토 예정
▶ 해외VC (투자사)
① 해외VC 요건(필수)
• 해외VC의 본사 소재지가 ‘한국’이 아닌 ‘국외’
- 해외VC의 소재지는 요건검토 시 적정 여부 판단
• 모집공고일 기준으로 최근 3년 이내(2021.5.8. ~ 2024.5.7.)의 해외펀드 운용 실적 보유
• 전문인력(벤처캐피탈 및 액셀러레이터 등 투자 관련 경력 2년 이상) 1명 이상 보유
• (우대)‘한국’ 지사·사무소 보유 또는 글로벌 펀드 출자 VC
② 투자유치 조건(투자방식은 요건 검토 시 적정 여부 판단)
• 최근 3년 이내 글로벌 팁스 사업신청 창업기업에게 20만불 이상 투자
- 모집공고일 기준으로 투자금 납입일이 3년 이내(2021.5.8. ~ 2024.5.7.)
- 해외VC 투자금액의 총 합산액(국내VC의 투자는 미인정)(외화기준)
- 신규 투자 또는 신규 및 기존지분 인수 등 혼합투자(단, 신규50% 이상) 모두 인정(금액기준)
- Co-GP의 경우, GP의 구성이 국내 VC로만 이루어진 경우 인정 불가
- USD 외 투자 시, 모집공고일 기준 서울외국환중개 매매기준율 미국달러(USD) 적용
- 2 -


In [23]:
visualize_query_retrieval_gt(first_qa, 1)

질문 : 어떤 분야의 기업들이 해외 VC 투자를 받을 수 있는 조건이나 지원을 받을 수 있는 프로그램에 대해 언급되고 있나요?
단락 내용:
1. ①팁스성공기업 ,시드포함)(프리
2. ②글로벌지원사업(글로벌액셀러레이팅,글로벌창업사관학교, 해외실증(PoC),KSC등)성공(졸업)기업
3. ③전략기술분야 ‧부품‧장비,10대초격차분야)붙임5‘해외VC글로벌’펀드(한국벤처투자출자)투자자
5. ⑤(해외VC)국내지사·사무소보유여부
6. ⑥(해외VC)투자전략및지원계획발표# 주요 참고사항*사업계획서양식:“[별첨1]사업계획서서식”참조**해외VC추천서:“[별첨2]증빙서류제출목록”참조***반드시동사업의사업계획서양식을활용 외양식제출시평가대상에서제외****가점관련증빙서류는사업신청시제출하며,미제출시가점불인정*****창업기업여부확인을위하여추가적인서류를요청할수있으며,기존에발급받은‘창업기업확인서’보유시해당서류로확인대체가능(단,사업공고일기준확인서상의유효기간이경과되지않았어야함)


### 도메인에 대한 정보를 추가해봅시다.

In [24]:
async def custom_query_gen(
		row: Dict,
		llm: BaseLLM,
		lang: str = "en",
) -> Dict:
	CUSTOM_PROMPT = {
		"ko": [
			ChatMessage(
				role=MessageRole.SYSTEM,
				content="""당신은 스타트업의 지원 프로그램에 대해 검색할 수 있는 RAG 시스템의 관리자입니다.
RAG 시스템을 관리를 위해, 사용자들의 예상 질문을 추출해야 합니다.
주어지는 문서는 다양한 스타트업 지원 프로그램에 대한 문서입니다.
문서를 보고 예상되는 사용자들의 질문을 적으세요.
""",
			)
		]
	}

	return await llama_index_generate_base(
		row, llm, CUSTOM_PROMPT[lang]
	)

In [25]:
second_qa = initial_qa.batch_apply(
	custom_query_gen,
	llm=llm,
	lang="ko",
)

[01/12/25 17:12:34] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=830882;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=677065;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[01/12/25 17:12:35] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=400524;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=52408;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=916833;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=591124;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

In [26]:
second_qa.data['query']

0    1. 해외VC 지원을 받기 위한 요건은 무엇인가요?\n2. 해외VC가 투자한 금액의...
1    1. 어떤 종류의 스타트업이 지원을 받을 수 있나요?\n2. 글로벌 창업 지원 프로...
2    1. 창업기업 선정 평가는 어떤 절차로 이루어지나요?\n2. 선정 평가의 일정과 결...
Name: query, dtype: object

In [27]:
visualize_query_retrieval_gt(second_qa, 0)

질문 : 1. 해외VC 지원을 받기 위한 요건은 무엇인가요?
2. 해외VC가 투자한 금액의 조건은 어떻게 되나요?
3. 해외VC의 본사 소재지가 한국이 아니어도 지원이 가능한가요?
4. 해외VC 투자금액의 총 합산액은 어떻게 계산되나요?
5. Co-GP의 경우, GP의 구성이 국내 VC로만 이루어진 경우 어떤 조건이 적용되나요?
단락 내용:
본점과 지점이 있는 경우 본점 사업자로만 신청 가능하며, 중복 신청한 경우 본점을 기준으로
요건검토 예정
▶ 해외VC (투자사)
① 해외VC 요건(필수)
• 해외VC의 본사 소재지가 ‘한국’이 아닌 ‘국외’
- 해외VC의 소재지는 요건검토 시 적정 여부 판단
• 모집공고일 기준으로 최근 3년 이내(2021.5.8. ~ 2024.5.7.)의 해외펀드 운용 실적 보유
• 전문인력(벤처캐피탈 및 액셀러레이터 등 투자 관련 경력 2년 이상) 1명 이상 보유
• (우대)‘한국’ 지사·사무소 보유 또는 글로벌 펀드 출자 VC
② 투자유치 조건(투자방식은 요건 검토 시 적정 여부 판단)
• 최근 3년 이내 글로벌 팁스 사업신청 창업기업에게 20만불 이상 투자
- 모집공고일 기준으로 투자금 납입일이 3년 이내(2021.5.8. ~ 2024.5.7.)
- 해외VC 투자금액의 총 합산액(국내VC의 투자는 미인정)(외화기준)
- 신규 투자 또는 신규 및 기존지분 인수 등 혼합투자(단, 신규50% 이상) 모두 인정(금액기준)
- Co-GP의 경우, GP의 구성이 국내 VC로만 이루어진 경우 인정 불가
- USD 외 투자 시, 모집공고일 기준 서울외국환중개 매매기준율 미국달러(USD) 적용
- 2 -


하나씩만 생성하도록 합니다